<p align="center">
<img src="https://github.com/datacamp/string-manipulation-in-sql-live-session/blob/master/assets/datacamp.svg?raw=True" alt = "DataCamp icon" width="50%">
</p>
<br><br>

## **PostgreSQL Live Training Template**

Welcome to the DataCamp "String Manipulation in SQL", where we will be covering a deep dive into a variety of techiques you can use to manipulate sting data in your SQL queries. Through a hands on coding session you will learn how to use built-in functions [...] to answer our real world business questions.

In today's notebook, you will learn:

- Explore a dataset in a SQL database by examining table structure and data types
- Converting or casting common data types
- How to manipulate string data using built-in string and character functions
- Create temporary tables and use them to simplify complex queries
- Learn how to use temporary tables to create new fields from unstructured string data
- Create user defined functions that can be used to simplify data manipulation tasks

## **The Dataset**

This data is taken from a .csv file called "short_term_rentals". As today's session will focus on creating queries. It contains the following columns.

- `id`: Unique identifier for rental ID
- `issue_date`: Date of Issue for the Short Term Rental certificate
- `status`: Application status.
- `location`: Rental Location
- `property_type`: Description of the building property type.
- `property_type_additional_description`: Additional building property type information.
- `condo_association`: Is the Short-Term Rental Unit part of a condominium association?
- `total_bedrooms`: Total number of legal bedrooms in the short-term rental unit
- `rented_bedrooms`: Number of legal bedrooms to be rented as part of this application.
- `maximum_renter_capacity`: Maximum number of people to which the short term rental will be rented.
- `kitchen`: Will the kitchen be available to the renter?
- `bathrooms`: Number of bathrooms available to the renter.
- `all_rental_services`: A single column list of all rental services used by this property.



## **Setting up PostgreSQL**

In [0]:
#@title **This block of code will install PosgreSQL**
%%capture
!wget -qO- https://www.postgresql.org/media/keys/ACCC4CF8.asc | apt-key add -
!echo "deb http://apt.postgresql.org/pub/repos/apt/ bionic-pgdg main" >/etc/apt/sources.list.d/pgdg.list
!apt -qq update
!apt -yq install postgresql-12 postgresql-client-12
!service postgresql start
# make calling psql shorter
!sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"  
!psql postgres -c "CREATE DATABASE root"  # now just !psql -c "..."
# load SQL extensions
%load_ext sql
%config SqlMagic.feedback=False 
%config SqlMagic.autopandas=True
%sql postgresql+psycopg2://@/postgres

In [0]:
#@title **This will download your data to local environment**
!wget -q https://raw.githubusercontent.com/datacamp/string-manipulation-in-sql-live-session/master/data/short_term_rentals.csv

In [4]:
#@title **This will create your table**
%%sql
-- Make sure to amend you table name, column names and types
DROP TABLE IF EXISTS rentals;
CREATE TABLE rentals(
 id int primary key,
 issue_date text,
 status text,
 location text,
 short_term_rental_type text,
 property_type text,
 property_type_additional_description text,
 condo_association text,
 total_bedrooms numeric,
 rented_bedrooms numeric,
 maximum_renter_capacity numeric,
 kitchen text,
 bathrooms numeric,
 all_rental_services text
);
COPY rentals
-- Make sure to point to correct file and delimiter 
FROM '/content/short_term_rentals.csv' DELIMITER ',' CSV HEADER;

 * postgresql+psycopg2://@/postgres


""


In [0]:
%%sql
-- SELECT first 10 rows

SELECT * 
FROM rentals
LIMIT 10;

# **Exploring the Dataset**

Now that we have PostgreSQL installed and our dataset loaded, let's take a closer look at the data that we'll be working with. 

In [0]:
%%sql

-- View the entire dataset

SELECT *
FROM rentals 

A few of things that you should notice from what we've seen so far.

1. **The dataset is much smaller than you would have expected with only 281 rows.** Let's think about this a bit. Do we believe it is correct?


2. **Looks like we have a date/time value `issue_date` that is stored as a `text` field rather than a date/time value.** This will likely cause unexpected results in our queries. For example:

In [0]:
%%sql

-- Sort the dataset by issue_date

SELECT *
FROM rentals
ORDER BY issue_date
LIMIT 5  

What did you notice about the results?  

Let's just confirm that the `issue_date` is indeed being sorted incorrectly by running the same query but sorting `DESCENDING` instead.

In [0]:
%%sql

-- Sort the dataset by issue_date

SELECT *
FROM rentals
ORDER BY issue_date DESC
LIMIT 5   

3. **The `all_rental_services` field contains a comma separated list of rental services.** This will mean will need to do some manipulation before doing any aggregation or analysis by rental service. 

4. **The `location` field contains all the address parts in a single column rather than individual fields like street address, city, state and postal code.** It also looks like there are lat/lon coordinates stored in this column as well which could become handy later if we want to visualize where the rentals are located across Cambridge.  For example:



In [0]:
%%sql

-- Show the ID,  location and all_rental_services columns

SELECT id, location, all_rental_services
FROM rentals
LIMIT 5   

## **What Have We Learned About the Short Term Rentals Data?**

So far, we have discovered:

*   There are 218 rentals in the dataset
*   The fields `issue_date`, `location` and `all_rental_services` will need to be converted or manipulated before we can get additional insights





## **Q&A**

# **Using built-in functions to manipulate string and character data**

Looking more closely at the Short Term Rentals dataset, there are at least three adjustments that need to be made to the data before we can get some meaningful insights.

1. First, the `issue_date` column needs to be converted to a date/time value before we can look at timeseries data.

2. Next, to get any aggregated insights about where in the city rental applications are most prevalent, we'll have to break apart the `location` column into it component address fields.

3. And finally, we'll need split the `all_rental_serivces` column so we can know who our competitors are in the city before we launch the pilot.

## **Converting data from one type to another using `CAST()`**

Let's get started by converting the `issue_date` to a date value.  Looking at the data as it's currently stored in the database, it appears that the column stores the month, day and year that the rental certification was issue but it's stored as a `TEXT` data type. 

The `CAST()` function converts the value of one data type to another.  In order to be able to use the `issue_date` in a query and get meaningful results, it needs to be converted from a `TEXT` data type to a `DATE` type and the `CAST()` can perform that operation. 

There does not appear to be any time of day associated with this column so we can just covert it to a `DATE` data type rather than a `TIMESTAMP` as we originally assumed.  

In [0]:
%%sql

-- Convert issue_date to a date

SELECT id, cast(issue_date AS date)
FROM rentals 
LIMIT 5


Now if we try to sort the dataset by the converted `issue_date` field, let's see if we get expected results.

In [0]:
%%sql

-- Convert issue_date to a date and sort ascending

SELECT id, cast(issue_date AS date)
FROM rentals 
--ORDER BY cast(issue_date AS date)
ORDER BY cast(issue_date AS date) DESC
LIMIT 5


## **Extracting string data using `SUBSTRING()` and `POSTIION()`**

Great work!  Now let's see what we can do with the `location` field.  Ideally, we would have columns for each address part including street address, postal code, etc. In order to do this we'll need to parse the `location` field into these parts. 

If we look at the data, we'll see that each part is sperated by a line break or `\n`. In PostgreSQL we can use `chr(10)` to find this character in a field value. 

This should make it easy to get what we need and there are a couple of techniques we can use to get there.

In [0]:
%%sql

-- Use substring() and position() functions to parse the location into various address values

SELECT substring(location, 0, position(chr(10) IN location)) AS street_address,
       substring(location, position(chr(10) IN location)+1, 19) AS city_state_zip,
       substring(location, position('(' IN location), position(')' IN location)) AS lat_lon
FROM rentals
LIMIT 5;


## **Extracting string data using `split_part()`**

THe `split_part()` function parses string data based on a specified delimiter into an `ARRAY` and allows access to each array index by specifying the index value as a parameter in the function.  The query above can be rewritten using the `split_part()` function which as you'll see actually simplifies the query.

In [0]:
%%sql

-- Use split_part() to parse the location into various address values

SELECT split_part(location, chr(10), 1) AS street_address, 
       split_part(location, chr(10), 2) AS city_state_zip, 
       split_part(location,chr(10), 3) AS lat_lon
FROM rentals
LIMIT 5;


## **Using `ARRAYS` to manipulate strings stored as comma-separated-values**

In [0]:
%%sql

-- Use string_to_array() to convert the all_rental_services from a comma-separated-list to an array

SELECT id, string_to_array(all_rental_services,',')
FROM rentals
LIMIT 5;


Now let's look closer at the result.  The comma-separated-list has been split into an `ARRAY` for each rental.  However, it appears that doing this produces `ARRAYS` with variable lenght which are going to be hard to work with later as we begin our analysis.

Ideally, we'd be able to have an indicator for each service to determine whether a rental was using that service.  Let's see if we can figure out how many service sare stored in the `all_rental_services` field.

PostgreSQL has a function called `unnest()` that will take an array and create unique rows for each index. Let's see what happens when we use the `unnest()` function on the result of `string_to_array()` for the first rental id 13727.

In [0]:
%%sql

-- Nest the string_to_array() function in unnest() to transpose the array into multiple unique rows.

SELECT id, unnest(string_to_array(all_rental_services,','))
FROM rentals
WHERE id = 13727;


The result of the `unnest()` function ened up creating two rows - one for Airbnb and one for HomeAway - both with id 13727.

Now we can get use these new skills to determine how many rentals are registered for each service by nesting the query above as a subquery and performing an aggregation to `COUNT()` the number of rental ids for each service. 

In [0]:
%%sql

-- Use the results from the unnest() function in the query

--SELECT count(id), services
SELECT count(id), trim(services)
FROM
(
    SELECT id, unnest(string_to_array(all_rental_services,','))
    FROM rentals
 ) as dt(id, services) 
-- GROUP BY services
GROUP BY trim(services)
ORDER BY count(id) DESC;
       


## **What Have We Learned About the Short Term Rentals Data?**

So far, we have discovered:

*   There rentals dataset contains data from March, 2018 through April, 2020.
*   The short term rentals market has about 22 different services.





## **Q&A**

## **Creating a temporary table with our rental services data**

Sometimes when you encountered data that needs to be transformed or manipulated before being usable in queries, you may decide that creating a temporary table with the result of your data manipulation will simplify your data analysis queries. Rather than creating complex queries with multiple subqueries or nested functions, you can create a temporary table and use it more efficiently.  Let's take a closer look

In [0]:
%%sql
DROP TABLE IF EXISTS rental_services;
-- Convert issue_date to a date and sort ascending
CREATE TABLE rental_services AS
  SELECT id, unnest(string_to_array(all_rental_services,',')) AS service
  FROM rentals;
SELECT *
FROM rental_services
LIMIT 5;
       


In [0]:
%%sql
DROP TABLE IF EXISTS address;
-- Convert issue_date to a date and sort ascending
CREATE TABLE address AS
  SELECT id, split_part(location, chr(10), 1) AS street_address, 
       right(split_part(location, chr(10), 2), 5) AS zip, 
       split_part(location,chr(10), 3) AS lat_lon
  FROM rentals;
SELECT *
FROM address
LIMIT 5;


# **Using temporary tables to simplify complex queries**

Once the temporary tables are created, your queries become much simpler and easier to manage, and you have the ability to run multiple queries against the temporary table which streamlines reuse.

In [0]:
%%sql
SELECT a.*, s.*
FROM address AS a INNER JOIN rental_services AS s ON a.id=s.id;

In [0]:
%%sql
SELECT a.zip, s.service, count(a.id) as total
FROM address AS a INNER JOIN rental_services AS s ON a.id=s.id
GROUP BY a.zip, s.service
ORDER BY zip, service;

# **User-defined functions to create reusable code**

In [0]:
%%sql
CREATE OR REPLACE FUNCTION getZipCode (rentalID int)
RETURNS varchar(5) AS $$
DECLARE zip varchar(5);
BEGIN
  SELECT right(split_part(location, chr(10), 2), 5) INTO zip
  FROM rentals
  WHERE id = rentalID;

  RETURN zip;
END;
$$ LANGUAGE plpgsql;

SELECT id, location, getZipCode(id) 
FROM rentals;


# **Putting it all together!**

Let's now use our new data manipulation skills to dig deeper into the short term rentals data to determine if Cambridge, MA is a good spot to pilot our new short term rentals app.

First, we'll see what the growth rate has been over the past couple of years with registrations.

Next we'll see what the most popular rental services are in the town.

And finally we'll take a look at which zip codes in the city have the largest number of rental units.  

Let's get started.

In [0]:
%%sql
SELECT extract(year FROM CAST(issue_date AS DATE)) AS year, count(id) 
FROM rentals
GROUP BY year
ORDER BY year;

In [0]:
%%sql
SELECT CASE WHEN trim(service) IN('Airbnb', 'HomeAway', 'VRBO', 'FlipKey', 
                                  'Craigslist', 'Boston_Rentals', 'Couch_Surfing') 
       THEN trim(service) ELSE 'Other' END as rentalService,
      count(r.id) as total
FROM rentals AS r INNER JOIN rental_services AS s ON r.id=s.id
GROUP BY rentalService
ORDER BY total DESC;


In [0]:
%%sql
SELECT getZipCode(r.id) AS zipCode, count(r.id) as total
FROM rentals AS r INNER JOIN rental_services AS s ON r.id=s.id
GROUP BY zipCode
ORDER BY zipCode;


# **Q&A**

# **Recap and Closing**

Thank you!